# 앙상블 RAG 검색 테스트

이 노트북은 BlindInsight의 앙상블 검색 시스템을 단계별로 테스트하고 조정할 수 있게 구현되어 있습니다.

## 주요 기능
1. **벡터 검색**: 코사인 유사도 기반 의미 검색
2. **BM25 검색**: 키워드 기반 검색
3. **앙상블 검색**: 벡터 + BM25 결합
4. **재랭킹**: Cross-encoder를 통한 결과 개선
5. **결과 비교**: 각 방법별 결과 비교 및 시각화

In [1]:
# 필요한 라이브러리 설치 및 임포트
import os
import sys
import asyncio
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# 프로젝트 경로 추가
project_path = r'C:\blind'
if project_path not in sys.path:
    sys.path.append(project_path)

# BlindInsight 모듈 임포트
try:
    from langchain.schema import Document
    from langchain_community.retrievers import BM25Retriever
    from langchain.retrievers import EnsembleRetriever
    from sentence_transformers import CrossEncoder
    
    from src.blindinsight.rag.embeddings import VectorStore
    from src.blindinsight.models.base import settings
    print("✅ 모든 모듈 임포트 완료")
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")
    print("requirements.txt의 패키지들이 설치되었는지 확인하세요.")

c:\blind\rag-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 모든 모듈 임포트 완료


c:\blind\rag-env\Lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
# 설정 및 초기화
print("🔧 설정 정보:")
print(f"- Vector DB 경로: {settings.vector_db_path}")
print(f"- OpenAI API 키 설정: {'✅' if settings.openai_api_key else '❌'}")
print(f"- 임베딩 모델: {settings.embedding_model}")

# VectorStore 초기화
vector_store = VectorStore(
    persist_directory=settings.vector_db_path,
)

print("✅ VectorStore 초기화 완료")

🔧 설정 정보:
- Vector DB 경로: C:/blind/data/embeddings
- OpenAI API 키 설정: ✅
- 임베딩 모델: text-embedding-3-small
ChromaDB 클라이언트 초기화 완료: C:/blind/data/embeddings
CompanyMetadataManager 초기화 완료: C:/blind/data/embeddings\company_metadata.db
컬렉션 'company_culture' 및 LangChain 래퍼 초기화 완료
컬렉션 'work_life_balance' 및 LangChain 래퍼 초기화 완료
컬렉션 'management' 및 LangChain 래퍼 초기화 완료
컬렉션 'salary_benefits' 및 LangChain 래퍼 초기화 완료
컬렉션 'career_growth' 및 LangChain 래퍼 초기화 완료
컬렉션 'general' 및 LangChain 래퍼 초기화 완료
✅ VectorStore 초기화 완료


## 1. 재랭킹 시스템 구현

기존 코드와 동일한 재랭킹 로직을 구현합니다.

In [3]:
class TestReRanker:
    """
    테스트용 재랭킹 클래스 - 기존 ReRanker와 동일한 로직
    """
    
    def __init__(self, model_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2"):
        """
        재랭커 초기화
        
        Args:
            model_name: 사용할 Cross-encoder 모델명
        """
        try:
            # Cross-encoder 모델 로드
            self.model = CrossEncoder(model_name)
            self.model_loaded = True
            print(f"✅ 재랭킹 모델 로드 완료: {model_name}")
        except Exception as e:
            print(f"❌ 재랭킹 모델 로드 실패: {str(e)}")
            self.model = None
            self.model_loaded = False
    
    def rerank_documents(
        self, 
        query: str, 
        documents: List[Document], 
        top_k: int = 10
    ) -> List[Tuple[Document, float]]:
        """
        문서들을 재랭킹하여 관련성 순으로 정렬
        
        Args:
            query: 검색 쿼리
            documents: 재랭킹할 문서 리스트
            top_k: 반환할 상위 문서 수
            
        Returns:
            (문서, 재랭킹_점수) 튜플 리스트
        """
        if not self.model_loaded or not documents:
            # 모델이 없으면 원본 순서 반환
            return [(doc, 0.5) for doc in documents[:top_k]]
        
        try:
            # 쿼리-문서 쌍 생성
            query_doc_pairs = [(query, doc.page_content) for doc in documents]
            
            # Cross-encoder로 관련성 점수 계산
            scores = self.model.predict(query_doc_pairs)
            
            # 점수와 문서를 묶어서 정렬
            doc_scores = list(zip(documents, scores))
            doc_scores.sort(key=lambda x: x[1], reverse=True)
            
            # 상위 k개 반환
            return doc_scores[:top_k]
            
        except Exception as e:
            print(f"❌ 재랭킹 실패: {str(e)}")
            return [(doc, 0.5) for doc in documents[:top_k]]

# 재랭커 초기화
reranker = TestReRanker()

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cpu


✅ 재랭킹 모델 로드 완료: cross-encoder/ms-marco-MiniLM-L-6-v2


## 2. 검색 결과 데이터 클래스

In [4]:
@dataclass
class TestSearchResult:
    """검색 결과를 담는 데이터 클래스"""
    
    document: Document           # 검색된 문서
    relevance_score: float       # 관련성 점수 (0-1)
    rank: int                   # 순위
    retrieval_method: str       # 검색 방법 (semantic, keyword, ensemble)
    metadata_scores: Dict[str, float] = None  # 메타데이터 기반 점수들
    
    def __post_init__(self):
        if self.metadata_scores is None:
            self.metadata_scores = {}

def result_to_document(result: Dict[str, Any]) -> Document:
    """검색 결과를 Document 객체로 변환"""
    return Document(
        page_content=result.get("content", ""),
        metadata=result.get("metadata", {})
    )

## 3. 앙상블 검색 구현

기존 코드의 `_ensemble_search` 메서드를 기반으로 구현합니다.

In [5]:
async def perform_vector_search(
    vector_store: VectorStore,
    query: str,
    collection_name: str,
    k: int,
    filters: Optional[Dict[str, Any]] = None
) -> List[Dict[str, Any]]:
    """벡터 데이터베이스에서 유사도 검색 실행"""
    print(f"🔍 벡터 검색 시작: {query[:50]}...")
    
    results = await vector_store.search_similar_documents(
        query=query,
        collection_name=collection_name,
        k=k,
        filter_dict=filters
    )
    
    print(f"✅ 벡터 검색 완료: {len(results)}개 결과")
    return results

async def perform_ensemble_search(
    vector_store: VectorStore,
    query: str,
    collection_name: str,
    k: int,
    filters: Optional[Dict[str, Any]] = None,
    bm25_weight: float = 0.2,
    vector_weight: float = 0.8
) -> List[Dict[str, Any]]:
    """
    Ensemble 검색 (BM25 + Vector Similarity)
    기존 _ensemble_search 메서드와 동일한 로직
    
    Args:
        vector_store: 벡터 저장소
        query: 검색 쿼리
        collection_name: 컬렉션명
        k: 반환할 문서 수
        filters: 메타데이터 필터
        bm25_weight: BM25 가중치
        vector_weight: 벡터 검색 가중치
        
    Returns:
        Ensemble 검색 결과 문서들
    """
    try:
        print(f"🔍 Ensemble 검색 시작: {query[:50]}...")
        print(f"   - BM25 가중치: {bm25_weight}, 벡터 가중치: {vector_weight}")
        
        # LangChain Chroma 래퍼 가져오기
        chroma = vector_store.get_langchain_chroma(collection_name)
        if not chroma:
            print(f"❌ LangChain Chroma 래퍼 없음, 폴백 검색 사용")
            return await perform_vector_search(vector_store, query, collection_name, k, filters)
        
        # ChromaDB에서 모든 문서 가져오기 (BM25용)
        raw_docs = chroma.get(include=["documents", "metadatas"])
        if not raw_docs["documents"]:
            print(f"❌ 컬렉션에 문서가 없음: {collection_name}")
            return []
        
        # Document 객체로 변환
        documents = [
            Document(page_content=doc, metadata=meta or {})
            for doc, meta in zip(raw_docs["documents"], raw_docs["metadatas"] or [{}] * len(raw_docs["documents"]))
        ]
        
        print(f"📚 BM25용 문서 로드: {len(documents)}개")
        
        # BM25 Retriever 생성
        bm25_retriever = BM25Retriever.from_documents(documents=documents, k=k)
        print(f"✅ BM25 Retriever 생성 완료")
        
        # Vector Search Retriever 생성
        search_kwargs = {'k': k}
        if filters:
            # ChromaDB 필터 형식으로 변환
            if len(filters) > 1:
                filter_conditions = []
                for key, value in filters.items():
                    filter_conditions.append({key: {"$eq": value}})
                search_kwargs['filter'] = {"$and": filter_conditions}
            else:
                key, value = next(iter(filters.items()))
                search_kwargs['filter'] = {key: {"$eq": value}}
        
        vector_retriever = chroma.as_retriever(
            search_type="similarity",
            search_kwargs=search_kwargs
        )
        print(f"✅ Vector Retriever 생성 완료")
        
        # Ensemble Retriever 생성
        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=[bm25_weight, vector_weight]
        )
        print(f"✅ Ensemble Retriever 생성 완료")
        
        # Ensemble 검색 실행
        ensemble_results = ensemble_retriever.get_relevant_documents(query)
        print(f"🎯 Ensemble 검색 결과: {len(ensemble_results)}개")
        
        # 결과를 기존 형식으로 변환 (랭킹 기반 유사도 점수)
        converted_results = []
        for i, doc in enumerate(ensemble_results[:k]):
            # 랭킹 기반으로 유사도 점수 계산 (1위: 0.95, 2위: 0.9, ..., 순차 감소)
            rank_based_score = max(0.1, 1.0 - (i * 0.05))
            converted_result = {
                "content": doc.page_content,
                "metadata": doc.metadata,
                "distance_score": rank_based_score,
                "ensemble_rank": i + 1,
                "method": "ensemble"
            }
            converted_results.append(converted_result)
        
        print(f"✅ Ensemble 검색 변환 완료: {len(converted_results)}개")
        return converted_results
        
    except Exception as e:
        print(f"❌ Ensemble 검색 실패: {str(e)}")
        # 폴백: 기존 벡터 검색 사용
        return await perform_vector_search(vector_store, query, collection_name, k, filters)

## 4. 통합 검색 함수

기존 `search` 메서드와 동일한 로직을 구현합니다.

In [6]:
async def integrated_search(
    vector_store: VectorStore,
    reranker: TestReRanker,
    query: str,
    collection_name: str = "company_culture",
    k: int = 20,
    filters: Optional[Dict[str, Any]] = None,
    search_type: str = "ensemble",
    enable_reranking: bool = True,
    bm25_weight: float = 0.2,
    vector_weight: float = 0.8
) -> List[TestSearchResult]:
    """
    통합 검색 실행 - 기존 RAGRetriever.search와 동일한 로직
    
    Args:
        vector_store: 벡터 저장소
        reranker: 재랭킹 모델
        query: 검색 쿼리
        collection_name: 검색할 컬렉션명
        k: 검색할 문서 수
        filters: 메타데이터 필터
        search_type: 검색 타입 (semantic, ensemble)
        enable_reranking: 재랭킹 활성화 여부
        bm25_weight: BM25 가중치
        vector_weight: 벡터 검색 가중치
        
    Returns:
        TestSearchResult 객체 리스트
    """
    
    print(f"\n🚀 통합 검색 시작")
    print(f"   - 쿼리: {query}")
    print(f"   - 컬렉션: {collection_name}")
    print(f"   - 검색 타입: {search_type}")
    print(f"   - 재랭킹: {enable_reranking}")
    print(f"   - 요청 문서 수: {k}")
    
    try:
        # 1단계: 검색 방법별 처리
        if search_type == "semantic":
            # 기본 벡터 검색
            raw_documents = await perform_vector_search(
                vector_store, query, collection_name, k * 2, filters
            )
            documents = raw_documents
            method = "semantic"
            
        elif search_type == "ensemble":
            # Ensemble 검색
            documents = await perform_ensemble_search(
                vector_store, query, collection_name, k, filters, bm25_weight, vector_weight
            )
            method = "ensemble"
        
        else:
            raise ValueError(f"지원하지 않는 검색 타입: {search_type}")
        
        if not documents:
            print("❌ 검색 결과가 없습니다.")
            return []
        
        print(f"✅ 1단계 {search_type} 검색 완료: {len(documents)}개")
        
        # 2단계: 재랭킹 (활성화된 경우)
        if enable_reranking and reranker.model_loaded:
            print(f"🔄 재랭킹 시작...")
            doc_list = [result_to_document(result) for result in documents]
            reranked = reranker.rerank_documents(query, doc_list, k)
            
            print(f"✅ 재랭킹 완료: {len(reranked)}개")
            
            # 재랭킹 결과를 TestSearchResult로 변환
            search_results = []
            scores = [score for _, score in reranked]
            
            if scores:
                min_score = min(scores)
                max_score = max(scores)
                score_range = max_score - min_score
                
            for rank, (doc, score) in enumerate(reranked):
                # 재랭킹 점수를 0~1 범위로 정규화
                if score_range > 0:
                    normalized_score = (score - min_score) / score_range
                    relevance_score = 0.1 + (normalized_score * 0.9)
                else:
                    relevance_score = 0.8
                
                search_results.append(TestSearchResult(
                    document=doc,
                    relevance_score=relevance_score,
                    rank=rank + 1,
                    retrieval_method=f"{method}_reranked",
                    metadata_scores={"rerank_score": float(score)}
                ))
        else:
            print(f"🔄 재랭킹 없이 상위 {k}개 선택")
            # 재랭킹 없이 상위 k개 반환
            search_results = []
            for rank, result in enumerate(documents[:k]):
                similarity_score = result.get("distance_score", 0.5)
                search_results.append(TestSearchResult(
                    document=result_to_document(result),
                    relevance_score=similarity_score,
                    rank=rank + 1,
                    retrieval_method=method,
                    metadata_scores={"original_score": similarity_score}
                ))
        
        print(f"🎯 최종 결과: {len(search_results)}개")
        return search_results
        
    except Exception as e:
        print(f"❌ 검색 실행 실패: {str(e)}")
        return []

## 5. 결과 시각화 및 비교 함수

In [7]:
def display_search_results(results: List[TestSearchResult], title: str = "검색 결과"):
    """검색 결과를 보기 좋게 표시"""
    print(f"\n{'='*80}")
    print(f"📊 {title}")
    print(f"{'='*80}")
    
    if not results:
        print("❌ 검색 결과가 없습니다.")
        return
    
    for result in results:
        print(f"\n🏆 순위: {result.rank} | 점수: {result.relevance_score:.3f} | 방법: {result.retrieval_method}")
        print(f"📄 내용: {result.document.page_content[:200]}...")
        
        if result.document.metadata:
            print(f"🏷️  메타데이터: {result.document.metadata}")
        
        if result.metadata_scores:
            print(f"📈 추가 점수: {result.metadata_scores}")
        
        print("-" * 80)

def create_results_dataframe(results_dict: Dict[str, List[TestSearchResult]]) -> pd.DataFrame:
    """검색 결과들을 DataFrame으로 변환"""
    data = []
    
    for method_name, results in results_dict.items():
        for result in results:
            data.append({
                'Method': method_name,
                'Rank': result.rank,
                'Score': result.relevance_score,
                'Content': result.document.page_content[:100] + "...",
                'Company': result.document.metadata.get('company', 'Unknown'),
                'Category': result.document.metadata.get('category', 'Unknown')
            })
    
    return pd.DataFrame(data)

def plot_score_comparison(results_dict: Dict[str, List[TestSearchResult]]):
    """검색 방법별 점수 분포 비교"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # 점수 분포 히스토그램
    for method_name, results in results_dict.items():
        scores = [r.relevance_score for r in results]
        axes[0].hist(scores, alpha=0.7, label=method_name, bins=10)
    
    axes[0].set_xlabel('Relevance Score')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('검색 방법별 점수 분포')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 순위별 점수 비교
    for method_name, results in results_dict.items():
        ranks = [r.rank for r in results]
        scores = [r.relevance_score for r in results]
        axes[1].plot(ranks, scores, marker='o', label=method_name, linewidth=2)
    
    axes[1].set_xlabel('Rank')
    axes[1].set_ylabel('Relevance Score')
    axes[1].set_title('순위별 점수 비교')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. 테스트 실행

이제 실제 검색을 테스트해보겠습니다.

In [11]:
# 테스트 설정
TEST_QUERY = "삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?"  # 원하는 쿼리로 변경하세요
COLLECTION_NAME = "work_life_balance"  # 사용할 컬렉션
K = 10  # 반환받을 문서 수
FILTERS = {"company": '삼성전자',"content_type":"pros"}
  # 필터가 있다면 설정 (예: {"company": "네이버"})

print(f"🔍 테스트 설정:")
print(f"   - 쿼리: {TEST_QUERY}")
print(f"   - 컬렉션: {COLLECTION_NAME}")
print(f"   - 문서 수: {K}")
print(f"   - 필터: {FILTERS}")

🔍 테스트 설정:
   - 쿼리: 삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?
   - 컬렉션: work_life_balance
   - 문서 수: 10
   - 필터: {'company': '삼성전자', 'content_type': 'pros'}


In [12]:
# 비동기 함수를 실행하기 위한 헬퍼
import asyncio

# 이벤트 루프가 이미 실행 중인 경우 처리
def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # Jupyter에서는 이미 이벤트 루프가 실행 중
            import nest_asyncio
            nest_asyncio.apply()
            return loop.run_until_complete(coro)
        else:
            return loop.run_until_complete(coro)
    except RuntimeError:
        return asyncio.run(coro)

# 필요한 경우 nest_asyncio 설치
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except ImportError:
    print("⚠️ nest_asyncio가 설치되지 않았습니다. 'pip install nest-asyncio'로 설치하세요.")

✅ nest_asyncio 적용 완료


## 7. 실제 검색 테스트

### 7.1 벡터 검색 테스트

In [13]:
# 벡터 검색 테스트
semantic_results = run_async(integrated_search(
    vector_store=vector_store,
    reranker=reranker,
    query=TEST_QUERY,
    collection_name=COLLECTION_NAME,
    k=K,
    filters=FILTERS,
    search_type="semantic",
    enable_reranking=False  # 재랭킹 없이 벡터 검색만
))

display_search_results(semantic_results, "벡터 검색 결과 (재랭킹 없음)")


🚀 통합 검색 시작
   - 쿼리: 삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?
   - 컬렉션: work_life_balance
   - 검색 타입: semantic
   - 재랭킹: False
   - 요청 문서 수: 10
🔍 벡터 검색 시작: 삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?...
[VectorStore] 검색 요청 - Collection: work_life_balance, Query: 삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?..., k: 20
[VectorStore] 필터: {'company': '삼성전자', 'content_type': 'pros'}
[VectorStore] 사용 가능한 컬렉션: ['company_culture', 'work_life_balance', 'management', 'salary_benefits', 'career_growth', 'general']
[VectorStore] 쿼리 임베딩 생성 중...
[VectorStore] 임베딩 생성 완료 (차원: 1536)
[VectorStore] 컬렉션 'work_life_balance' 문서 수: 27
[VectorStore] ChromaDB where 절: {'$and': [{'company': {'$eq': '삼성전자'}}, {'content_type': {'$eq': 'pros'}}]}
[VectorStore] ChromaDB 검색 실행 중...
[VectorStore] ChromaDB 응답 - documents: 9개
[VectorStore] 결과 포맷팅 시작...
[VectorStore] results 구조: <class 'dict'>
[VectorStore] results.keys(): dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas', 'distances'])
[VectorStore] documents 길이: 1
[VectorSt

### 7.2 앙상블 검색 테스트

In [14]:
# 앙상블 검색 테스트 (재랭킹 없음)
ensemble_results = run_async(integrated_search(
    vector_store=vector_store,
    reranker=reranker,
    query=TEST_QUERY,
    collection_name=COLLECTION_NAME,
    k=K,
    filters=FILTERS,
    search_type="ensemble",
    enable_reranking=False,  # 재랭킹 없이 앙상블만
    bm25_weight=0.2,
    vector_weight=0.8
))

display_search_results(ensemble_results, "앙상블 검색 결과 (재랭킹 없음)")


🚀 통합 검색 시작
   - 쿼리: 삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?
   - 컬렉션: work_life_balance
   - 검색 타입: ensemble
   - 재랭킹: False
   - 요청 문서 수: 10
🔍 Ensemble 검색 시작: 삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?...
   - BM25 가중치: 0.2, 벡터 가중치: 0.8
📚 BM25용 문서 로드: 27개
✅ BM25 Retriever 생성 완료
✅ Vector Retriever 생성 완료
✅ Ensemble Retriever 생성 완료


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_34368\3167032502.py:101: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ensemble_results = ensemble_retriever.get_relevant_documents(query)
100%|██████████| 1/1 [00:00<00:00,  2.66it/s]

🎯 Ensemble 검색 결과: 16개
✅ Ensemble 검색 변환 완료: 10개
✅ 1단계 ensemble 검색 완료: 10개
🔄 재랭킹 없이 상위 10개 선택
🎯 최종 결과: 10개

📊 앙상블 검색 결과 (재랭킹 없음)

🏆 순위: 1 | 점수: 1.000 | 방법: ensemble
📄 내용: 부바부 케바케지만 첫직장이 여기면 이만한데 없다는걸 모르고 퇴사하고 후회할수있다.
자율 출퇴근, 자유로운 휴가, 꼰대꼰대 거려도 LGD 비교하면 스마트하고 깔끔하고 젠틀하다....
🏷️  메타데이터: {'company': '삼성전자', 'chunk_index': 0, 'review_date': 'unknown', 'position': 'unknown', 'content_type': 'pros', 'rating_numeric': 0.0, 'employee_status': '현직원', 'source_type': 'company_chunk', 'sentiment_label': 'neutral', 'chunk_id': 'work_life_balance_pros_삼성전자_0019_00', 'content_length': 97, 'processing_timestamp': '', 'sentiment_score': 0.0, 'rating_level': 'unknown', 'collection_target': 'work_life_balance', 'category': 'work_life_balance'}
📈 추가 점수: {'original_score': 1.0}
--------------------------------------------------------------------------------

🏆 순위: 2 | 점수: 0.950 | 방법: ensemble
📄 내용: 자율 출퇴근제 최고 (부서마다 보직장에 따라 눈치 보는곳도 있음) 사내 도보시 휴대폰 사용을 금지하여 안전사고 방지함.
코로나로 인해 30프로 인원이 3교대로 재택근무중. (전체의 1 3이 아니라 전체 3

### 7.3 앙상블 + 재랭킹 테스트

In [16]:
# 앙상블 + 재랭킹 테스트
ensemble_reranked_results = run_async(integrated_search(
    vector_store=vector_store,
    reranker=reranker,
    query=TEST_QUERY,
    collection_name=COLLECTION_NAME,
    k=K,
    filters=FILTERS,
    search_type="ensemble",
    enable_reranking=True,  # 재랭킹 활성화
    bm25_weight=0.7,
    vector_weight=0.3
))

display_search_results(ensemble_reranked_results, "앙상블 + 재랭킹 검색 결과")


🚀 통합 검색 시작
   - 쿼리: 삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?
   - 컬렉션: work_life_balance
   - 검색 타입: ensemble
   - 재랭킹: True
   - 요청 문서 수: 10
🔍 Ensemble 검색 시작: 삼성전자의 워라밸은 어떄? 업무가 너무 많지는 않나?...
   - BM25 가중치: 0.7, 벡터 가중치: 0.3
📚 BM25용 문서 로드: 27개
✅ BM25 Retriever 생성 완료
✅ Vector Retriever 생성 완료
✅ Ensemble Retriever 생성 완료


100%|██████████| 1/1 [00:00<00:00,  3.55it/s]


🎯 Ensemble 검색 결과: 16개
✅ Ensemble 검색 변환 완료: 10개
✅ 1단계 ensemble 검색 완료: 10개
🔄 재랭킹 시작...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]

✅ 재랭킹 완료: 10개
🎯 최종 결과: 10개

📊 앙상블 + 재랭킹 검색 결과

🏆 순위: 1 | 점수: 1.000 | 방법: ensemble_reranked
📄 내용: 자율 출퇴근제 최고 (부서마다 보직장에 따라 눈치 보는곳도 있음) 사내 도보시 휴대폰 사용을 금지하여 안전사고 방지함.
코로나로 인해 30프로 인원이 3교대로 재택근무중. (전체의 1 3이 아니라 전체 30프로 인원 중 1 3이 돌아가며 재택)...
🏷️  메타데이터: {'content_length': 136, 'chunk_id': 'work_life_balance_pros_삼성전자_0022_00', 'rating_numeric': 0.0, 'processing_timestamp': '', 'position': 'unknown', 'chunk_index': 0, 'company': '삼성전자', 'collection_target': 'work_life_balance', 'content_type': 'pros', 'rating_level': 'unknown', 'category': 'work_life_balance', 'review_date': 'unknown', 'sentiment_label': 'neutral', 'employee_status': '현직원', 'source_type': 'company_chunk', 'sentiment_score': 0.0}
📈 추가 점수: {'rerank_score': 8.611920356750488}
--------------------------------------------------------------------------------

🏆 순위: 2 | 점수: 0.881 | 방법: ensemble_reranked
📄 내용: 부바부 케바케지만 첫직장이 여기면 이만한데 없다는걸 모르고 퇴사하고 후회할수있다.
자율 출퇴근, 자유로운 휴가, 꼰대꼰대 거려도 LGD 비교하면 스마트하고 깔끔하고 젠틀하다....
🏷️  메타데이터: {'sentiment_l

## 8. 결과 비교 및 분석

In [ ]:
# 결과 비교를 위한 딕셔너리
results_comparison = {
    "Semantic Only": semantic_results,
    "Ensemble": ensemble_results,
    "Ensemble + Rerank": ensemble_reranked_results
}

# DataFrame 생성
df_results = create_results_dataframe(results_comparison)
print("📊 검색 결과 비교표")
display(df_results.head(20))  # 상위 20개만 표시

In [ ]:
# 점수 분포 시각화
plot_score_comparison(results_comparison)

In [ ]:
# 통계 요약
print("📈 검색 방법별 통계 요약")
print("="*80)

for method_name, results in results_comparison.items():
    if results:
        scores = [r.relevance_score for r in results]
        print(f"\n🔍 {method_name}:")
        print(f"   - 결과 수: {len(results)}")
        print(f"   - 평균 점수: {np.mean(scores):.3f}")
        print(f"   - 최고 점수: {np.max(scores):.3f}")
        print(f"   - 최저 점수: {np.min(scores):.3f}")
        print(f"   - 표준편차: {np.std(scores):.3f}")
    else:
        print(f"\n🔍 {method_name}: 결과 없음")

## 9. 파라미터 조정 테스트

BM25와 벡터 검색의 가중치를 조정하여 결과를 비교해보겠습니다.

In [ ]:
# 다양한 가중치 설정 테스트
weight_configurations = [
    {"bm25_weight": 0.1, "vector_weight": 0.9, "name": "Vector Heavy (0.1:0.9)"},
    {"bm25_weight": 0.2, "vector_weight": 0.8, "name": "Default (0.2:0.8)"},
    {"bm25_weight": 0.3, "vector_weight": 0.7, "name": "Balanced (0.3:0.7)"},
    {"bm25_weight": 0.5, "vector_weight": 0.5, "name": "Equal (0.5:0.5)"},
    {"bm25_weight": 0.7, "vector_weight": 0.3, "name": "BM25 Heavy (0.7:0.3)"},
]

weight_test_results = {}

print("🧪 가중치 조정 테스트 시작...")
for config in weight_configurations:
    print(f"\n테스트 중: {config['name']}")
    
    results = run_async(integrated_search(
        vector_store=vector_store,
        reranker=reranker,
        query=TEST_QUERY,
        collection_name=COLLECTION_NAME,
        k=K,
        filters=FILTERS,
        search_type="ensemble",
        enable_reranking=True,
        bm25_weight=config["bm25_weight"],
        vector_weight=config["vector_weight"]
    ))
    
    weight_test_results[config["name"]] = results

print("✅ 가중치 조정 테스트 완료")

In [ ]:
# 가중치 테스트 결과 시각화
plot_score_comparison(weight_test_results)

# 가중치별 통계
print("📊 가중치별 성능 비교")
print("="*80)

weight_stats = []
for config_name, results in weight_test_results.items():
    if results:
        scores = [r.relevance_score for r in results]
        weight_stats.append({
            'Configuration': config_name,
            'Avg Score': np.mean(scores),
            'Max Score': np.max(scores),
            'Min Score': np.min(scores),
            'Std Dev': np.std(scores),
            'Results Count': len(results)
        })

df_weight_stats = pd.DataFrame(weight_stats)
display(df_weight_stats)

## 10. 사용자 정의 테스트

이 섹션에서 원하는 쿼리와 설정으로 테스트를 진행하세요.

In [ ]:
# 🔧 여기서 설정을 변경하세요
CUSTOM_QUERY = "카카오 워라밸 야근"  # 원하는 쿼리 입력
CUSTOM_COLLECTION = "general"         # 컬렉션 이름
CUSTOM_K = 15                        # 반환받을 문서 수
CUSTOM_FILTERS = None                # 필터 (예: {"company": "카카오"})
CUSTOM_BM25_WEIGHT = 0.3             # BM25 가중치
CUSTOM_VECTOR_WEIGHT = 0.7           # 벡터 가중치
ENABLE_CUSTOM_RERANKING = True       # 재랭킹 사용 여부

print(f"🎯 사용자 정의 테스트 설정:")
print(f"   - 쿼리: {CUSTOM_QUERY}")
print(f"   - 컬렉션: {CUSTOM_COLLECTION}")
print(f"   - 문서 수: {CUSTOM_K}")
print(f"   - 필터: {CUSTOM_FILTERS}")
print(f"   - BM25 가중치: {CUSTOM_BM25_WEIGHT}")
print(f"   - 벡터 가중치: {CUSTOM_VECTOR_WEIGHT}")
print(f"   - 재랭킹: {ENABLE_CUSTOM_RERANKING}")

In [ ]:
# 사용자 정의 테스트 실행
custom_results = run_async(integrated_search(
    vector_store=vector_store,
    reranker=reranker,
    query=CUSTOM_QUERY,
    collection_name=CUSTOM_COLLECTION,
    k=CUSTOM_K,
    filters=CUSTOM_FILTERS,
    search_type="ensemble",
    enable_reranking=ENABLE_CUSTOM_RERANKING,
    bm25_weight=CUSTOM_BM25_WEIGHT,
    vector_weight=CUSTOM_VECTOR_WEIGHT
))

display_search_results(custom_results, f"사용자 정의 테스트 결과: {CUSTOM_QUERY}")

## 📝 결론 및 다음 단계

이 노트북을 통해 다음을 확인할 수 있습니다:

1. **벡터 검색**: 의미적 유사성 기반 검색 성능
2. **앙상블 검색**: BM25 + 벡터 검색 결합 효과
3. **재랭킹**: Cross-encoder를 통한 결과 개선 정도
4. **가중치 조정**: BM25와 벡터 검색 비율에 따른 성능 변화

### 추가 실험 아이디어:
- 다른 재랭킹 모델 테스트
- 컬렉션별 성능 비교
- 다양한 쿼리 타입에 대한 성능 분석
- 필터 적용에 따른 성능 변화

### 성능 최적화 팁:
- 짧고 구체적인 쿼리에는 BM25 가중치를 높이세요
- 의미적 검색이 중요한 경우 벡터 가중치를 높이세요
- 재랭킹은 정확도를 높이지만 속도가 느려집니다
- 필터를 적절히 활용하면 관련성이 높은 결과를 얻을 수 있습니다